In [3]:
from pkg_resources import resource_filename
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import chess
matplotlib.style.use('ggplot')

In [82]:
# Import stockfish values and fens
relative_filepath = 'data/extracted_data/'
filepath = resource_filename('guerilla', relative_filepath + 'sf_values.nsv')
sf = pd.read_csv(filepath, names=['values_rnd'], header=None)

filepath = resource_filename('guerilla', relative_filepath + 'fens.nsv')
sf['fens_rnd'] = pd.read_csv(filepath, header=None)

filepath = resource_filename('guerilla', relative_filepath + 'sf_values_old.nsv')
sf['values'] = pd.read_csv(filepath, header=None)

filepath = resource_filename('guerilla', relative_filepath + 'fens_old.nsv')
sf['fens'] = pd.read_csv(filepath, header=None)

In [13]:
# Add numbering (SLOW)
start_fen = 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'
sf['move_num'] = int(0)

for row in range(len(sf)):
    if sf.ix[row, 'fens'] == start_fen:
        continue
    else:
        sf.ix[row, 'move_num'] = sf.ix[row - 1, 'move_num'] + 1

In [19]:
sf.rename(columns={'move_count':'move_num'}, inplace=True)

In [83]:
#values.plot.hist(bins=100)
hist, bins = np.histogram(sf['values'], bins = 100)
plt.bar(bins[:-1], hist*1.0 / hist.sum(), width=(bins[1]-bins[0]))
plt.xticks(np.arange(0, 1.1, 0.1))
#plt.xlim([-2000, 2000])
plt.title('Stockfish Score Distribution')
plt.xlabel('P(Win)')
plt.ylabel('Normalized # of fens')
plt.show()

In [85]:
sf[['values','values_rnd']].plot.hist(bins=100, stacked = False, alpha = 0.75 )
plt.show()

In [31]:
g = sf.groupby('move_num', axis=0)
mean_data = g.aggregate(np.mean)
var_data = g.aggregate(np.var)

In [79]:
#g.size().plot()
plt.ylabel('Number of Games')
plt.xlabel('Move Number')
plt.show()

In [71]:
mean_data.plot()
for col in mean_data.columns:
    top = mean_data[col] + var_data[col]
    bot = mean_data[col] - var_data[col]
    
    plt.fill_between(mean_data.index, top, bot,
                     color = '#348ABD' if col == 'values' else '#A60628'
                     , alpha=0.25)

plt.xlabel('Move Number')
plt.ylabel('P(Win)')
plt.show()

In [80]:
var_data.plot()
plt.show()